# Running CrimeRank

### Functions

In [21]:
def grad_calc_on(y, s, k):
    
    N = len(y)
    lambda_ = np.zeros((N, 1))
    in_or_out = np.zeros((N, 1))

    idx = np.argsort(s, axis=0)[::-1]
    idx = np.argsort(idx, axis=0)

    for i in range(N):
        if idx[i] <= k:
            in_or_out[i] = 1

    for i in range(N):
        for j in range(1, N):
            if (y[i] != y[j]) & (in_or_out[i] != in_or_out[j]):
                DZ = y[i] - y[j]

                if y[i] > y[j]:
                    rho = 1 / (1 + np.exp(s[i] - s[j]))
                    lambda_[i] = lambda_[i] + rho*np.abs(DZ)
                    lambda_[j] = lambda_[j] - rho*np.abs(DZ)
                else:
                    rho = 1 / (1 + np.exp(s[j] - s[i]))
                    lambda_[i] = lambda_[i] - rho*np.abs(DZ)
                    lambda_[j] = lambda_[j] + rho*np.abs(DZ)

    return lambda_

In [22]:
def PAI_error(y, s, k):

    n_days = len(y)
    e = np.zeros((n_days, 1))
    crime = np.zeros((n_days, 1))
    y_top_k = np.zeros((k, 1))

    for i in range(n_days):
        idx = np.argsort(s[i], axis=0)[::-1]
        ys = y[i]
        ys = ys[idx]
        if len(y_top_k) != len(ys[:k]):
            continue
        else:
            y_top_k = y_top_k + ys[:k] 
            e[i] = sum(ys[:k])
            crime[i] = sum(ys)
        
    crimes_top_k = sum(e)
    PAI = crimes_top_k / sum(crime)
    y_top_k = np.sort(y_top_k)[::-1]

    return PAI, y_top_k

In [23]:
from shapely.geometry import Polygon
def check_intersection(box_1, box_2):

    poly_1 = np.asarray([k for k in zip(box_1[0], box_1[1])])
    poly_1 = Polygon(poly_1)
    
    poly_2 = np.asarray([k for k in zip(box_2[0], box_2[1])])
    poly_2 = Polygon(poly_2)

    if poly_1.intersects(poly_2):
        return 1
    else:
        return 0   

In [26]:
# pip install ismember
from ismember import ismember
def sort_greedy(cell_locations, s, k):

    N = len(cell_locations)
    cell_temp = cell_locations
    cell_idx = np.argsort(s, axis=0)[::-1]
    cell_locations = np.asarray(cell_locations)[cell_idx[:N]] # ordered by score
    s = s[cell_idx[:N]] # ordered score
    cell_locations_2 = []
    cell_locations_2.append(cell_locations[0])

    count = 1
    j = 0
    while count < k:
        output = np.ones((count, 1))
        while max(output) == 1:
            j += 1
            for i in range(count):
                output[i] = check_intersection(np.squeeze(cell_locations_2[i]), 
                                                np.squeeze(cell_locations[j]))
                if output[i] == 1:
                    break

        cell_locations_2.append(cell_locations[j])
        count += 1

    top_locations = cell_locations_2
    c1 = np.zeros((len(cell_temp), 8))
    c2 = np.zeros((len(top_locations), 8))

    for i in range(len(cell_temp)):
        c1[i] = [k for k in cell_temp[i][0]] + [k for k in cell_temp[i][1]]
    
    for i in range(len(top_locations)):
        c2[i] = [k for k in np.squeeze(top_locations[i])[0]] + \
                [k for k in np.squeeze(top_locations[i])[1]]

    top_indices = ismember(c1, c2, 'rows')
    top_indices = np.flatnonzero(top_indices[0])
    
    return top_locations, top_indices

In [27]:
def PAI_error_off(y, s, k, cell_locations):

    n_days = len(y)
    e = np.zeros((n_days,1))
    crime = np.zeros((n_days,1))
    y_top_k = np.zeros((k, 1))

    for i in range(n_days):
        top_locations, top_indices = sort_greedy(cell_locations, s[i], k)
        ys = y[i][top_indices]
        y_top_k = y_top_k + ys[:k]
        e[i] = np.sum(ys)
        crime[i] = np.sum(y[i])
        
    crimes_top_k = np.sum(e)
    PAI = crimes_top_k / np.sum(crime)
    y_top_k = np.sort(y_top_k)[::-1]

    return PAI, y_top_k, top_locations

### Running the algorithm

In [103]:
train_n_days = len(Y_train)
test_n_days = len(Y_test)

for i in range(train_n_days):
    Y_train[i] = Y_train[i][X_train[i][-1] / np.max(X_train[i][-1]) > 0.01]
    for j in range(4):
        X_train[i][j] = X_train[i][j][X_train[i][-1] / np.max(X_train[i][-1]) > 0.01]

S = []
for i in range(train_n_days):
    S.append(np.random.randn(len(Y_train[i]), 1))

St = []
for i in range(test_n_days):
    St.append(np.random.randn(len(Y_test[i]), 1))

N1 = np.zeros((train_n_days, 1))
for i in range(train_n_days):
    N1[i] = len(Y_train[i])

XTR = np.zeros((int(sum(N1)), len(X_train[0])))
YTR = np.zeros((int(sum(N1)), 1))


XTR[:int(N1[0])] = np.asarray(X_train[0]).T
YTR[:int(N1[0])] = np.reshape(np.asarray(Y_train[0]).T, (-1,1))
for i in range(1, train_n_days):
    start = int(sum(N1[:i]))
    end = int(sum(N1[:i+1]))
    XTR[start: end] = np.asarray(X_train[i]).T
    YTR[start: end] = np.reshape(np.asarray(Y_train[i]).T, (-1,1))

In [ ]:
import copy
from sklearn.tree import DecisionTreeRegressor

n_iter = 500
k = 100
total_area_city = 1485*1000*1000
total_cell_area = k*cell_size**2
lambda_days = copy.deepcopy(S)
lambda_ = np.zeros((int(sum(N1)), 1))
PAI_test = np.zeros((n_iter, 1))
PAI_train = np.zeros((n_iter, 1))
Y_top_test = []
Y_top_train = []
top_locations = []
learning_rate = 0.00025

for i in range(n_iter):

    for j in range(train_n_days):
        lambda_days[j] = grad_calc_on(Y_train[j], S[j], k)  
    
    lambda_[:int(N1[0])] = lambda_days[0]
    for j in range(1, train_n_days):
        start = int(sum(N1[:j]))
        end = int(sum(N1[:j+1]))
        lambda_[start: end] = lambda_days[j]

    sample = int(sum(N1))
    size = int(np.ceil(sample/4))
    index = np.random.choice(sample, size, replace=False)
    
    regressor = DecisionTreeRegressor(random_state=0, 
                                    min_samples_leaf=100,
                                    ccp_alpha=0.015)
    tree = regressor.fit(XTR[index], lambda_[index])

    for j in range(train_n_days):
        x_train = np.concatenate([np.reshape(X_train[j][k], (-1,1)) 
                                        for k in range(4)], axis=1)
        S[j] = S[j] + np.reshape(learning_rate * tree.predict(x_train), (-1,1))

    for j in range(test_n_days):
        St[j] = St[j] + np.reshape(learning_rate * 
                                    tree.predict(np.squeeze(X_test[j]).T), (-1,1))

    
    PAI, y = PAI_error(Y_train, S, k)
    PAI_train[i] = PAI / (total_cell_area / total_area_city)
    Y_top_train.append(y)

    PAI, y, locations = PAI_error_off(Y_test, St, k, cell_locations_test)
    PAI_test[i] = PAI / (total_cell_area / total_area_city)
    Y_top_test.append(y)
    top_locations.append(locations)

    print(i)

In [102]:
with open('Y_train', 'rb') as fp:
    Y_train = pickle.load(fp)

with open('X_train', 'rb') as fp:
    X_train = pickle.load(fp)

with open('X_test', 'rb') as fp:
    X_test = pickle.load(fp)

with open('Y_test', 'rb') as fp:
    Y_test = pickle.load(fp)